In [6]:
import crypto

contract_address = "0xd4964Df4dc2eb6B2fD4157DFda264AA9dd720C92" # PeanutV2
dev_address = "0x6B3751c5b04Aa818EA90115AA06a4D9A36A16f02"
usdc_contract_address = "0xD87Ba7A50B2E7E660f678A895E4B72E7CB4CCd9C" # USDC
uni_contract_address = "0x1f9840a85d5aF5bf1D1762F925BDADdC4201F984" # UNI

In [7]:
contract, web3 = crypto.get_contract(address=contract_address, abi=crypto.ABI, network="ethereum-goerli")

In [8]:
print(f"contract owner: {contract.functions.owner().call()}")
print(f"balance: {web3.eth.getBalance(contract_address)}")
print(f"deposit count: {contract.functions.getDepositCount().call()}")

contract owner: 0x6B3751c5b04Aa818EA90115AA06a4D9A36A16f02
balance: 0
deposit count: 0


In [37]:
# make an ether deposit 
# reference:
#      * @dev Function to make a deposit
#      * @param _tokenAddress address of the token being sent. 0x0 for eth
#      * @param _contractType uint8 for the type of contract being sent. 0 for eth, 1 for erc20, 2 for erc721, 3 for erc1155
#      * @param _amount uint256 of the amount of tokens being sent (if erc20)
#      * @param _tokenId uint256 of the id of the token being sent if erc721 or erc1155
#      * @param _key bytes32 of the hash of the deposit password.
#      * @param _lockCost uint256 of the cost in wei to lock the deposit to claimer
#      * @param _senderCanWithdraw bool of whether the sender can withdraw the deposit
#      * @param _ownerCanWithdraw bool of whether the owner can withdraw the deposit
#      * @return uint256 of the index of the deposit

pwd = "test"
pwd_hash = crypto.hash_password(pwd)


txn = contract.functions.makeDeposit(
    web3.toChecksumAddress("0x0000000000000000000000000000000000000000"), # token address
    0,
    0,
    0,
    pwd_hash,
    0,
    True,
    True,
).buildTransaction(
    {
        "value": web3.toWei(0.0000167, "ether"),
        "gas": 200000,
        "gasPrice": web3.eth.gasPrice,
        "nonce": web3.eth.getTransactionCount(web3.eth.defaultAccount),
    }
)

signed_txn = web3.eth.account.sign_transaction(txn, private_key=crypto.ADMIN_ACCOUNT_KEY)
tx_hash = web3.eth.sendRawTransaction(signed_txn.rawTransaction)
tx_receipt = web3.eth.waitForTransactionReceipt(tx_hash)


# print gas used
print(f"gas used: {tx_receipt.gasUsed}")
print(f"gas price: {web3.eth.gasPrice}")
print(f"gas cost: {web3.eth.gasPrice * tx_receipt.gasUsed}")
print(f"tx total cost: {crypto.estimate_cost(tx_receipt.gasUsed, 'ethereum', web3)}")


# get deposit index from event
deposit_index = tx_receipt.logs[0].data
# turn from hex to int
deposit_index = int(deposit_index, 16)
print(f"deposit index: {deposit_index}")


gas used: 148497
gas price: 1020
gas cost: 151466940
gas: 148497, gas_price: 1020, eth_price: 1272.05000000
tx total cost: 1.92673521027e-07
deposit index: 0


In [13]:
# tx cost
crypto.estimate_cost(tx_receipt.gasUsed, "ethereum", web3)

gas: 131403, gas_price: 1018, eth_price: 1271.01000000


1.7002078851654e-07

In [10]:
print(f"contract owner: {contract.functions.owner().call()}")
print(f"balance: {web3.eth.getBalance(contract_address)}")
print(f"deposit count: {contract.functions.getDepositCount().call()}")

contract owner: 0x6B3751c5b04Aa818EA90115AA06a4D9A36A16f02
balance: 0
deposit count: 1


In [9]:
# now try to claim the deposit (withdrawDepositOwner())
# inputs: int256 _index, address _recipient)

txn = contract.functions.withdrawOwner(
    deposit_index,
    web3.toChecksumAddress("0xd3222f01b4154528ceC2807D99385d0fA4473A31"),
).buildTransaction(
    {
        "gas": 200000,
        "gasPrice": web3.eth.gasPrice,
        "nonce": web3.eth.getTransactionCount(web3.eth.defaultAccount),
    }
)

signed_txn = web3.eth.account.sign_transaction(txn, private_key=crypto.ADMIN_ACCOUNT_KEY)
tx_hash = web3.eth.sendRawTransaction(signed_txn.rawTransaction)
tx_receipt = web3.eth.waitForTransactionReceipt(tx_hash)

print(tx_receipt)   

AttributeDict({'blockHash': HexBytes('0x1dfdcd043c64719ed50ccfa511fde6e64fabfaf9a079572e6bc2aa4de2a5184f'), 'blockNumber': 8066584, 'contractAddress': None, 'cumulativeGasUsed': 6842381, 'effectiveGasPrice': 1016, 'from': '0x6B3751c5b04Aa818EA90115AA06a4D9A36A16f02', 'gasUsed': 56077, 'logs': [AttributeDict({'address': '0xeCDB0A54881fd12146BA257Ab37dcdbDB85aACD5', 'blockHash': HexBytes('0x1dfdcd043c64719ed50ccfa511fde6e64fabfaf9a079572e6bc2aa4de2a5184f'), 'blockNumber': 8066584, 'data': '0x00000000000000000000000000000000000000000000000000003ca9cdce7800', 'logIndex': 115, 'removed': False, 'topics': [HexBytes('0x884edad9ce6fa2440d8a54cc123490eb96d2768479d49ff9c7366125a9424364'), HexBytes('0x000000000000000000000000d3222f01b4154528cec2807d99385d0fa4473a31')], 'transactionHash': HexBytes('0xcbe0a26ac9ab2d20a8d885d59815443e804e7ecc671d3b995c28bdf19600feef'), 'transactionIndex': 56})], 'logsBloom': HexBytes('0x00000000000000000000000000000000000000000000000000100000000000000000000000000000

In [11]:
# print amount of deposits
print(contract.functions.getDepositCount().call())

# print each deposit
for i in range(contract.functions.getDepositCount().call()):
    print(contract.functions.getDeposit(i).call())
    

1
('0x0000000000000000000000000000000000000000', 0, 0, 0, '0x0000000000000000000000000000000000000000', b'\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00', '0x0000000000000000000000000000000000000000', 0, 0, False, False)


In [39]:
# now try to claim the deposit (withdrawDepositOwner())
# inputs: int256 _index, address _recipient)

txn = contract.functions.withdrawDepositOwner(
    deposit_index,
    web3.toChecksumAddress("0xd3222f01b4154528ceC2807D99385d0fA4473A31"),
).buildTransaction(
    {
        "gas": 200000,
        "gasPrice": web3.eth.gasPrice,
        "nonce": web3.eth.getTransactionCount(web3.eth.defaultAccount),
    }
)

signed_txn = web3.eth.account.sign_transaction(txn, private_key=crypto.ADMIN_ACCOUNT_KEY)
tx_hash = web3.eth.sendRawTransaction(signed_txn.rawTransaction)
tx_receipt = web3.eth.waitForTransactionReceipt(tx_hash)

print(tx_receipt)

AttributeDict({'blockHash': HexBytes('0x0e3fd0601b8aa0b6e800c149dc1ba0482b046ddeed2e1582d89c345fdc2bf346'), 'blockNumber': 8066552, 'contractAddress': None, 'cumulativeGasUsed': 9935019, 'effectiveGasPrice': 1026, 'from': '0x6B3751c5b04Aa818EA90115AA06a4D9A36A16f02', 'gasUsed': 28725, 'logs': [], 'logsBloom': HexBytes('0x00000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000'), 'status': 0, 'to': '0x7105b1fB681ba40316806E8CE08F715aD4165d5A', 'transactionHash': HexBytes('0xdb1cec6368ef60dea503ce18c7fab275b3078faaae46e5f056a1b082dcb42db9'

In [20]:
# now lets make an erc20 deposit
# reference:
#      * @dev Function to make a deposit
#      * @param _tokenAddress address of the token being sent. 0x0 for eth
#      * @param _contractType uint8 for the type of contract being sent. 0 for eth, 1 for erc20, 2 for erc721, 3 for erc1155
#      * @param _amount uint256 of the amount of tokens being sent (if erc20)
#      * @param _tokenId uint256 of the id of the token being sent if erc721 or erc1155
#      * @param _key bytes32 of the hash of the deposit password.
#      * @param _lockCost uint256 of the cost in wei to lock the deposit to claimer
#      * @param _senderCanWithdraw bool of whether the sender can withdraw the deposit
#      * @param _ownerCanWithdraw bool of whether the owner can withdraw the deposit
#      * @return uint256 of the index of the deposit

pwd = "test"
pwd_hash = crypto.hash_password(pwd)

# send 0.008 uni tokens to the contract (token has 18 decimals)
txn = contract.functions.makeDeposit(
    web3.toChecksumAddress(uni_contract_address), # USDC on goerli token address
    1,
    int(0.008 * 10**18),   
    0,
    pwd_hash,
    0,
    True,
    True,
).buildTransaction(
    {
        "value": web3.toWei(0.0, "ether"),
        "gas": 200000,
        "gasPrice": web3.eth.gasPrice,
        "nonce": web3.eth.getTransactionCount(web3.eth.defaultAccount),
    }
)

signed_txn = web3.eth.account.sign_transaction(txn, private_key=crypto.ADMIN_ACCOUNT_KEY)
tx_hash = web3.eth.sendRawTransaction(signed_txn.rawTransaction)
tx_receipt = web3.eth.waitForTransactionReceipt(tx_hash)
print(tx_receipt.transactionHash.hex())
print(f"tx_status: {tx_receipt.status}")
if tx_receipt.status == 0:
    print("tx failed")
    # print why it failed
    print(web3.eth.getTransactionReceipt(tx_hash).logs[0].data)

# print gas used
print(f"gas used: {tx_receipt.gasUsed}")
print(f"gas price: {web3.eth.gasPrice}")
print(f"gas cost: {web3.eth.gasPrice * tx_receipt.gasUsed}")
print(f"tx total cost: {crypto.estimate_cost(tx_receipt.gasUsed, 'ethereum', web3)}")

# get deposit index from event
deposit_index = tx_receipt.logs[-1].data
# turn from hex to int
deposit_index = int(deposit_index, 16)
print(f"deposit index: {deposit_index}")


0xcd7e6ebec29b83c34c445ee45aa57c983e201e3de9e05e88bd86ca353204d323
tx_status: 1
gas used: 192349
gas price: 1010
gas cost: 194272490
gas: 192349, gas_price: 1010, eth_price: 1268.35000000
tx total cost: 2.464055126915e-07
deposit index: 926336713898529563388567880069503262826159877325124512315660672063305037119488000000000000001


In [17]:
# fetch a reverted transaction:
tx = web3.eth.get_transaction(tx_receipt.transactionHash.hex())
# build a new transaction to replay:
replay_tx = {
    'to': tx['to'],
    'from': tx['from'],
    'value': tx['value'],
    'data': tx['input'],
}

# replay the transaction locally and print revert reason:
try:
    web3.eth.call(replay_tx, tx.blockNumber - 1)
except Exception as e: 
    print(e)


execution reverted: INSUFFICIENT ALLOWANCE


In [12]:
# print all deposits
print(f"deposit count: {contract.functions.getDepositCount().call()}")
for i in range(contract.functions.getDepositCount().call()):
    print(contract.functions.getDeposit(i).call())
    

deposit count: 1
('0x1f9840a85d5aF5bf1D1762F925BDADdC4201F984', 1, 1, 0, '0x6B3751c5b04Aa818EA90115AA06a4D9A36A16f02', b'\xba\xa7c\x83:ro\x12+\xe4\x9c\xd7\x13\xda\xfb\xe7%N\x04{\xa19\xe6?r\xa5\xf1;\xb4m\x02\xd6', '0x0000000000000000000000000000000000000000', 0, 0, True, True)
